# 当代人工智能Project3

## Neural Machine Translation

In [ ]:
!pip install datasets transformers sacrebleu sentencepiece numpy pandas matplotlib
# 安装huggingface的数据集接口，transformer等库

     |████████████████████████████████| 290 kB 10.9 MB/s 
     |████████████████████████████████| 3.1 MB 26.8 MB/s 
     |████████████████████████████████| 90 kB 8.9 MB/s 
     |████████████████████████████████| 1.2 MB 45.7 MB/s 
     |████████████████████████████████| 243 kB 48.1 MB/s 
     |████████████████████████████████| 132 kB 54.4 MB/s 
     |████████████████████████████████| 59 kB 6.3 MB/s 
     |████████████████████████████████| 1.1 MB 44.4 MB/s 
     |████████████████████████████████| 3.3 MB 41.5 MB/s 
     |████████████████████████████████| 596 kB 39.6 MB/s 
     |████████████████████████████████| 895 kB 44.0 MB/s 
     |████████████████████████████████| 192 kB 52.2 MB/s 
     |████████████████████████████████| 271 kB 50.8 MB/s 
     |████████████████████████████████| 160 kB 49.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import datasets
import random
import pandas as pd
import numpy as np
from datasets import load_metric
from IPython.display import display, HTML

使用Huggingface的Transformer库


In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer

In [ ]:
from datasets import load_dataset
en_de_raw_datasets = load_dataset("wmt16", "de-en")
# 加载wmt16数据集，选择英德数据

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.69k [00:00<?, ?B/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/658M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset wmt16 downloaded and prepared to /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/0d9fb3e814712c785176ad8cdb9f465fbe6479000ee6546725db30ad8a8b5f8a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

打印一些文本数据，看一下数据的格式

In [ ]:
en_de_raw_datasets["train"][0]

{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode',
  'en': 'Resumption of the session'}}

In [ ]:
def random_elements(dataset, num_examples=k):
    k = 5 #展示数据的条数
    assert num_examples <= len(dataset), "Failed."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
random_elements(en_de_raw_datasets["train"])

,translation
0,"{'de': 'Sobald ein Land die Konvertierbarkeit in Gold abschaffte, was es in der Lage, einen Kurs der Geldmengenexpansion zu verfolgen, was tendenziell zu einem Ende der Deflation führte.', 'en': 'Once a country eliminated convertibility into gold, it was free to pursue monetary expansion, and deflation tended to end.'}"
1,"{'de': 'Es kann nicht sein, daß lose abgepackte Lebensmittel nicht gekennzeichnet werden müssen.', 'en': 'It is unacceptable that loose foodstuffs need not be labelled.'}"
2,"{'de': 'Die Läufer, klassiert vom 61. bis zum letzten Platz nach zwei Etappen, starten deshalb in Lacanau morgens um 7 Uhr 30.', 'en': 'The runners ranking from the 61st to the last position after 2 stages will, therefore, start out from Lacanau at 7h30 in the morning.'}"
3,"{'de': 'Einige Kilometer von Colle di Val d’Elsa entfernt, in der Nähe der Quelle des Flusses Elsa.', 'en': 'A few kilometres from Colle di Val d'Elsa, close to the springs of the river Elsa.'}"
4,"{'de': '(Beifall)', 'en': '(Applause)'}"


加载了Helsinki-NLP/opus-mt-en-de的BERT模型用于实验

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
model_checkpoint = "Helsinki-NLP/opus-mt-en-de" 
# 加载一个wmt比赛已经预训练好的模型
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/750k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

In [ ]:
with tokenizer.as_target_tokenizer():
    # print(tokenizer("Hello world!"))
    model_input = tokenizer("hello tokens welcome to neural machine translation!") # tokenize
    tokens = tokenizer.convert_ids_to_tokens(model_input['input_ids'])
    # special tokens
    print('tokens: {}'.format(tokens))

{'input_ids': [8152, 166, 2, 60, 32, 18, 7795, 46, 2066, 68, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
tokens: ['▁Hell', 'o', ',', '▁this', '▁on', 'e', '▁sen', 't', 'ence', '!', '</s>']


考虑到Colab能提供的资源有限，这里限制max len为64

In [ ]:
max_input_length = 64
max_target_length = 64
source_lang = "en" # 原始语言为英语
target_lang = "de" # 目标语言为德语

In [ ]:
prefix = ""

def data_preprocess(lang_examples):
    inputs = [prefix + ex[source_lang] for ex in lang_examples["translation"]]
    targets = [ex[target_lang] for ex in lang_examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = en_de_raw_datasets.map(data_preprocess, batched=True)

  0%|          | 0/4549 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets.save_to_disk('Con_ai_course_project3')

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to("cuda") # 使用Google Colab 的 GPU训练

Downloading:   0%|          | 0.00/284M [00:00<?, ?B/s]

In [ ]:
batch_size = 8 # 根据资源调整
args = Seq2SeqTrainingArguments(
    "Con_ai_course_project3",
    # evaluation_strategy = "epoch", # 不能用epoch来eval，因为训不完一个epoch
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    load_best_model_at_end=True,     # load best model，尽管在colab不可能训练完
    logging_steps=500,               # K步保存一次
    evaluation_strategy="steps",     # evaluate
    fp16=False,
)
print('batchsize:',batch_size)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


batchsize: 8


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

使用sacreBLEU对翻译结果进行衡量，直接照搬官方仓库里的使用方法即可

In [ ]:

metric = load_metric("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) # 若无法decode则用一个大的负值替代
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running training *****
  Num examples = 4548885
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1705833


Step,Training Loss,Validation Loss,Bleu,Gen Len
500,1.717800,1.365783,29.457100,26.800400
1000,1.721900,1.382770,28.724500,26.660700
1500,1.750600,1.398325,28.701600,26.703600
2000,1.982500,1.395685,28.676300,26.573500
2500,2.124100,1.389101,28.901900,26.628900
3000,2.178300,1.390327,29.059200,26.602100
3500,2.098000,1.383869,29.190100,26.712300
4000,2.054300,1.395263,28.687300,26.676800
4500,2.110500,1.377900,29.381300,26.749200
5000,2.162300,1.380088,29.134200,26.729400


The following columns in the evaluation set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running Evaluation *****
  Num examples = 2169
  Batch size = 8
Saving model checkpoint to test-translation/checkpoint-500
Configuration saved in test-translation/checkpoint-500/config.json
Model weights saved in test-translation/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-translation/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-translation/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation.
***** Running Evaluation *****
  Num examples = 2169
  Batch size = 8
Saving model checkpoint to test-translation/checkpoint-1000
Configuration saved in test-translation/checkpoint-1000/config.json
Model weights saved in test-translation/checkpoint-1000/pytorch_model.bin
t

Step,Training Loss,Validation Loss,Bleu,Gen Len
500,1.717800,1.365783,29.457100,26.800400
1000,1.721900,1.382770,28.724500,26.660700
1500,1.750600,1.398325,28.701600,26.703600
2000,1.982500,1.395685,28.676300,26.573500
2500,2.124100,1.389101,28.901900,26.628900
3000,2.178300,1.390327,29.059200,26.602100
3500,2.098000,1.383869,29.190100,26.712300
4000,2.054300,1.395263,28.687300,26.676800
4500,2.110500,1.377900,29.381300,26.749200
5000,2.162300,1.380088,29.134200,26.729400
